In [1]:
import os
import glob
from pymed_paperscraper import PubMed #pip install pymed-paperscraper
import re
import csv
import datetime

# Outil pour "automatiser" la collecte des doi sur Pubmed

On suit la procédure décrite par l'Université de Lorraine dans https://gitlab.com/Cthulhus_Queen/barometre_scienceouverte_universitedelorraine/-/raw/master/requetes_bdd.txt?ref_type=heads

L'objectif est d'automatiser la collecte et l'enregistrement des publications sur Pubmed selon l'organisation des dossiers  <span style="color:red">(Data > raw > dossier par année)</span> pour que le code fonctionne.

On utilise le module "pymed" pour interroger l'API de PubMed : https://github.com/jannisborn/pymed

Pour le nettoyage des fichiers, on utilisera ensuite le notebook **01_nettoyage_donnees.ipynb**

**Note sur pymed-paperscraper** : <span style="color:red">`pymed-paperscraper`est un fork du `pymed` archivé depuis 2000. Les dernières mises à jour de pymed-paperscrapper date de 2023...</span> Toutefois, nous avons utilisé ce module, car c'est celui qui a été le simple d'utilisation pour nous parmi les nombreux modules python qui proposent de faciliter la navigation dans l'API de PubMed.

In [2]:
pubmed = PubMed(tool="MyTool", email="aymeric.luneau02@univ-paris8.fr") # ?

On indique le nom de l'université que l'on souhaite utiliser pour le nommage des fichiers. Par exemple :

- "univ_paris8" pour Université Paris 8 - Saint-Denis
- "lorraine" pour université de Lorraine



In [3]:
nom_universite = "univ_paris8" #pour le nom des fichiers



## Définition de la requête

On commence par définir la requête de base en précisant les différentes formes utilisées par les auteurs pour nommer leur affiliation. Exemples des intitutlés utilisés pour l'université Paris 8 :

`
(("universite de paris VIII"[Affiliation] OR
"universite de paris-VIII"[Affiliation] OR 
"universite paris-VIII"[Affiliation] OR
"universite paris-8-saint-denis"[Affiliation] OR
"universite paris-8"[Affiliation] OR
"universite de paris 8"[Affiliation] OR
"universite paris 8"[Affiliation] OR
"university of paris-8"[Affiliation] OR
"university of paris 8"[Affiliation] OR
university of paris VIII"[Affiliation])
`

Pour commencer, il est conseillé de construire la requête à partir du site de PubMed : https://pubmed.ncbi.nlm.nih.gov/advanced/

Il suffit ensuite de remplacer la requête dans la cellule ci-dessous par la vôtre.
On pourra ainsi s'assurer que les résultats obtenus à l'aide du script ci-après correspond à ceux trouver sur le site.


In [4]:
requete_de_base = '''
"universite de paris VIII"[Affiliation] OR
"universite de paris-VIII"[Affiliation] OR
"universite paris-VIII"[Affiliation] OR
"universite paris-8"[Affiliation] OR
"universite de paris 8"[Affiliation] OR
"universite paris 8"[Affiliation] OR
"university of paris-8"[Affiliation] OR
"university of paris 8"[Affiliation] OR
"university of paris VIII"[Affiliation] OR
"university paris VIII"[Affiliation] OR
"university paris 8"[Affiliation]
''' #COLLER VOTRE REQUETE ENTRE les trois guillemets

Enfin, la boucle qui permet de récupérer les données. <span style="color:red">**La période de temps prise en compte est comprise entre l'année 2016 (incluse) et l'année en cours (exlue)**</span>. "Exclue" signifie que les articles publiés pendant l'année en cours (exemple : 2025) ne seront pas récupérés.

Le script effectue les actions suivantes :

0. Pour chaque année comprise entre 2016 et année en cours exlue 

1. Vérifie que les dossiers existent. Correspond au point 5 du Readme sur https://gitlab.com/Cthulhus_Queen/barometre_scienceouverte_universitedelorraine
    * Si le dossier existe, il est vidé de son contenu
    * Si le dossier n'existent pas, il est créé. Permettra par exemple de créer le dossier `./Data/raw/2025`, puis `./Data/raw/2026`, etc.

2. On crée la requete complète
    * À la requête de base, on ajoute la borne de temps (exemple : `2016/01/01:2016/12/31[Date - Publication]`)
    * On interroge PubMed

3. On crée un fichier dans le dossier de l'année requêté selon la norme indiquée par l'université de Lorraine
    * Exemple : `"./Data/raw/2016/pubmed_univ_paris8_2016.csv"`
    * Ce fichier `.csv`contient les colonnes suivante : "DOI", "PMID", "title", "auteurs", "pub_date", "Publication year"

4. Pour chaque article identifié, on récupère les données correspondant aux colonnes ci-dessus

5. On enregistre le résultat dans le fichier `.csv` (voir point 3)


**Quelque remarque sur les résultats retournés par le scrapper :**
Dans certain cas, on obtient le pmid et le doi de l'article ainsi que ceux des références citées dans l'article. Dans ce cas on choisi de ne garder que les premières valeurs. Par contre il peut être utile de conserver les autres, dans le cas d'une étude sur les réseaux de co-citations

`pubmed__paperscraper` prend la date "pubmed" qui peut varier de la date de publication. Pour avoir un exemple, allez sur la page suivante https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=28039490, faites `Ctrl+f` et cherchez "PubMedPubDate PubStatus". On constate que:

* `<PubMedPubDate PubStatus="pubmed">`donne 2017
* `<PubMedPubDate PubStatus="accepted">` donne 2016

De manière générale, plusieurs métadonnées correspondent à des dates qui varient parfois à l'instar de la publication n°26348557 : `<PubMedPubDate PubStatus="received">`, `<PubMedPubDate PubStatus="pubmed">`, `<ArticleDate>`, etc.

```xml
<PubMedPubDate PubStatus="received">
<Year>2015</Year>
<Month>1</Month>
<Day>7</Day>
</PubMedPubDate>
<PubMedPubDate PubStatus="revised">
<Year>2015</Year>
<Month>8</Month>
<Day>19</Day>
</PubMedPubDate>
<PubMedPubDate PubStatus="accepted">
<Year>2015</Year>
<Month>8</Month>
<Day>22</Day>
</PubMedPubDate>
<PubMedPubDate PubStatus="entrez">
<Year>2015</Year>
<Month>9</Month>
<Day>9</Day>
<Hour>6</Hour>
<Minute>0</Minute>
</PubMedPubDate>
<PubMedPubDate PubStatus="pubmed">
<Year>2015</Year>
<Month>9</Month>
<Day>9</Day>
<Hour>6</Hour>
<Minute>0</Minute>
</PubMedPubDate>
<PubMedPubDate PubStatus="medline">
<Year>2016</Year>
<Month>9</Month>
<Day>8</Day>
<Hour>6</Hour>
<Minute>0</Minute>
</PubMedPubDate>
```

On a donc choisi de définir l'année itérée (variable `year`) comme "Publication year" (variable utilisé ensuite dans le notebook **01_nettoyage_donnees.ipynb**).

In [5]:
for year in range(2016,datetime.date.today().year):
## 1. Vérifie que les dossiers existent.
    # Si le dossier existe, il est vidé de son contenu
    if os.path.exists(f"./Data/raw/{year}"):
        files = glob.glob(f"./Data/raw/{year}/*.csv")
        for f in files:
            os.remove(f)
    # Si le dossier n'existent pas, il est créé
    else:
        os.makedirs(f"./Data/raw/{year}")
    
## 2. On crée la requete complète
    query = f'(({requete_de_base}) AND {year}/01/01:{year}/12/31[Date - Publication])' # correspond à '(({}) AND{}/01/01:{}/12/31[Date - Publication])'.format(requete_de_base, str(year), str(year))
    print(query)
    results = pubmed.query(query)
    compteur = 0 # pour compter le nombre d'articles trouvés

## 3. On crée un fichier dans le dossier de l'année requêté selon la norme indiquée par l'université de Lorraine
    fieldnames = ["DOI", "PMID", "title", "auteurs", "pub_date", "Publication year", "published_date", "pymed_publication_year"] # On peut ajouter des colonnes ou en enlever selon son envie
    name_file = f"./Data/raw/{year}/pubmed_{nom_universite}_{year}.csv"
    with open(name_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

## 4. Pour chaque article identifié, on récupère les données correspondant aux colonnes ci-dessus
    for n, article in enumerate(results):
        compteur += 1

        # Extract and format information from the article
        article_id = article.pubmed_id
        # Dans certain cas, article.pubmed_id donne à la fois le pmid de l'article et celui des références citées dans l'article qui ont aussi un pmid. Dans ce cas on choisi de ne garder que le premier. Par contre il peut être utile de conserver les autres, dans le cas d'une étude sur les réseaux de co-citations
        if re.search('\n', article_id):
            article_id = article_id.split('\n')[0]
        else:
            pass
        title = article.title
        doi = article.doi

        # Dans certain cas, article.doi donne à la fois le doi de l'article et celui des références citées dans l'article. Dans ce cas on choisi de ne garder que le premier. Par contre il peut être utile de conserver les autres, dans le cas d'une étude sur les réseaux de co-citations
        if doi is not None :
            if re.search('\n', doi):
                doi = doi.split('\n')[0]
            else:
                pass
        else:
            pass

        #published_date = article.published_date

        publication_date = article.publication_date
        #publication_date_r = article.publication_date_received

        # pubmed__paperscraper prend la date "pubmed" qui peut varier de la date de publication. Exemple : https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=28039490, il y a <PubMedPubDate PubStatus="pubmed"> = 2017 et <PubMedPubDate PubStatus="accepted"> = 2016
        # En attendant une proposition de modification du code de pubmed_paperscraper, on choisi de définir l'année itérée comme Publication year
        pymed_publication_year = publication_date.year 
        publication_year = year
        authors = article.authors
        list_auteur = []
        for x in authors:
            nom_auteur = f"{x['firstname']}_{x['lastname']}"
            list_auteur.append(nom_auteur)
            
    
        # Show information about the article, décommenter la ligne ci-dessous si on veut afficher les résultats
        #print(article_id, doi, publication_date, publication_year, pymed_dpublication_year)
        
## 5. On enregistre le résultat dans le fichier `.csv` (voir point 3)
        row = {"DOI": doi, "PMID":article_id, "title":title, "auteurs": "|".join(list_auteur), "pub_date" : publication_date, "Publication year" : publication_year, "pymed_publication_year":pymed_publication_year }
        with open(name_file, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)
    print(year, compteur)

    

    

((
"universite de paris VIII"[Affiliation] OR
"universite de paris-VIII"[Affiliation] OR
"universite paris-VIII"[Affiliation] OR
"universite paris-8"[Affiliation] OR
"universite de paris 8"[Affiliation] OR
"universite paris 8"[Affiliation] OR
"university of paris-8"[Affiliation] OR
"university of paris 8"[Affiliation] OR
"university of paris VIII"[Affiliation] OR
"university paris VIII"[Affiliation] OR
"university paris 8"[Affiliation]
) AND 2016/01/01:2016/12/31[Date - Publication])
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/esearch.fcgi?tool=MyTool&email=aymeric.luneau02%40univ-paris8.fr&db=pubmed&term=%28%28%0A%22universite+de+paris+VIII%22%5BAffiliation%5D+OR%0A%22universite+de+paris-VIII%22%5BAffiliation%5D+OR%0A%22universite+paris-VIII%22%5BAffiliation%5D+OR%0A%22universite+paris-8%22%5BAffiliation%5D+OR%0A%22universite+de+paris+8%22%5BAffiliation%